In [56]:
import logging
from PIL import Image
from glob import glob
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF
from torchvision import transforms
import random
import os
from pathlib import Path
import xarray as xr
import rioxarray as rioxr

In [3]:
data_dir = Path("/Users/gordonlogie/Documents/Projects/Neural_Earth/SOW_2/data/xview_building_damage/geotiffs/hold")
img_dir = Path(data_dir).joinpath("images")

In [22]:
img_suffix_list = list(img_dir.glob('*_post_disaster.tif'))
img_suffix_list = [img_suffix.name.split('_post_disaster.tif')[0]  for img_suffix in img_suffix_list]

In [89]:
select_index = 0

img_suffix = img_suffix_list[select_index]

pre_img = xr.open_rasterio(img_dir.joinpath(f"{img_suffix}_pre_disaster.tif")) / 255.0
post_img = xr.open_rasterio(img_dir.joinpath(f"{img_suffix}_post_disaster.tif")) / 255.0

/usr/local/Caskroom/miniforge/base/envs/ms-building-damage/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  """
/usr/local/Caskroom/miniforge/base/envs/ms-building-damage/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  


In [98]:
def process_image(img, normalize=True):

    img_mean = img.mean(dim=["x", "y"]).values.tolist()
    img_std = img.std(dim=["x", "y"]).values.tolist()

    if normalize:
        norm_func = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=img_mean, std=img_std)
        ])
    else:
        norm_func = transforms.ToTensor()

    img_arr = np.transpose(np.array(img), (1, 2, 0)).astype(dtype='float64')
    img_norm = norm_func(img_arr)

    return img_norm

pre_img_norm = process_image(pre_img)
post_img_norm = process_image(post_img)

In [110]:
img_suffix

'hurricane-harvey_00000306'

# Testing dataset creation

In [111]:
import logging
# from PIL import Image
from glob import glob
import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF
from torchvision import transforms
import random
import os
from pathlib import Path
import xarray as xr
import rioxarray as rioxr

class DisasterDataset(Dataset):
    def __init__(self, data_dir, img_suffix_list, transform:bool, normalize:bool):
        
        self.data_dir = data_dir
        self.transform = transform
        self.normalize = normalize
        self.img_suffix_list = img_suffix_list
    
    def __process_image__(self, img):

        img_mean = img.mean(dim=["x", "y"]).values.tolist()
        img_std = img.std(dim=["x", "y"]).values.tolist()

        if self.normalize:
            norm_func = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=img_mean, std=img_std)
            ])
        else:
            norm_func = transforms.ToTensor()

        img_arr = np.transpose(np.array(img), (1, 2, 0)).astype(dtype='float64')
        img_norm = norm_func(img_arr)

        return img_norm
    
    def __getitem__(self, i):
        
        img_dir = Path(self.data_dir).joinpath("images")

        img_suffix = self.img_suffix_list[i]

        pre_img = xr.open_rasterio(img_dir.joinpath(f"{img_suffix}_pre_disaster.tif")) / 255.0
        post_img = xr.open_rasterio(img_dir.joinpath(f"{img_suffix}_post_disaster.tif")) / 255.0

        pre_img = self.__process_image__(pre_img)
        post_img = self.__process_image__(post_img)

        return {'pre_image': pre_img, 'post_image': post_img}